# Example showcase of PyRep and Panda Robot without ROS

In [1]:
import numpy as np
from pyrep import PyRep
from pyrep.objects.shape import Shape
from pyrep.const import PrimitiveShape, TextureMappingMode
from pyrep.robots.arms.panda import Panda
from pyrep.robots.end_effectors.panda_gripper import PandaGripper
from pyrep.errors import ConfigurationPathError

Load Scenes/Textures directory

In [2]:
import os
#SCENES='/'.join(os.path.realpath(__file__).split('/')[:-2])+str('/include/scenes/')
#TEXTURES='/'.join(os.path.realpath(__file__).split('/')[:-2])+str('/include/textures/')
SCENES=os.path.realpath("./../include/scenes/")
TEXTURES=os.path.realpath("./../include/textures/")

In [3]:
SCENE = 'scene_panda_franka_gripper.ttt'

In [4]:
pr = PyRep()
# Launch the application with a scene file
pr.launch(SCENES+'//'+SCENE, headless=False, responsive_ui=True)
pr.start()  # Start the simulation
pr.step()

In [5]:
panda = Panda()  # Get the panda from the scene
gripper = PandaGripper()  # Get the panda gripper from the scene

In [6]:
velocities = [.1, .2, .3, .4, .5, .6, .7]
panda.set_joint_target_velocities(velocities)
panda.set_joint_positions([0.,0.,0.,-np.pi/2,0.,np.pi/3,np.pi])

In [7]:
panda.set_joint_target_positions([0.,0.,0.,0.,0.,0.,0.])
for i in range(200):
    pr.step() # simulate behaviour

In [8]:
panda.set_joint_positions([0.,0.,0.,0.,0.,0.,0.])

for i in range(200):
    pr.step() # simulate behaviour

In [9]:
panda.get_joint_count()
panda.get_joint_forces()
panda.get_joint_modes()
[panda.joints[j].is_motor_enabled() for j in range(7)]
panda.joints[0].set_joint_target_position(1.)
panda.set_joint_target_velocities([0.2,0.02,0.02,0.02,0.02,0.02,0.02])
panda.set_joint_target_positions([0.,-np.pi/4,0.,-np.pi/2,0.,np.pi/3,np.pi])

In [10]:
for i in range(200):
    pr.step() # simulate behaviour

# Add some objects to scene

In [11]:
colors = [[1.,0.,0.], [0.,1.,0.], [0.,0.,1.]]
positions = [[0.3, 0.3, 0.1], [-0.5, -0.3, 0.1], [0.2, -0.3, 0.1]]
shapes = [PrimitiveShape.CYLINDER, PrimitiveShape.CONE, PrimitiveShape.SPHERE]
for i in range(0,3):
    object = Shape.create(type=shapes[i],
                          color=colors[i], size=[0.075, 0.075, 0.075],
                          position=positions[i])

In [12]:
for i in range(20):
    pr.step() # simulate behaviour

# Move the gripper 

In [13]:
done = False
# Open the gripper halfway at a velocity of 0.04.
while not done:
    done = gripper.actuate(0.0, velocity=0.04)
    pr.step()

# Control Panda via IK

In [27]:
DELTA = 0.01
starting_joint_positions = panda.get_joint_positions()
pos, quat = panda.get_tip().get_position(), panda.get_tip().get_quaternion()

def move(index, delta):
    pos[index] += delta
    new_joint_angles = panda.solve_ik_via_jacobian(pos, quaternion=quat)
    panda.set_joint_target_positions(new_joint_angles)
    pr.step()

[move(2, DELTA) for _ in range(10)]
[move(2, -DELTA) for _ in range(10)]
[move(1, -DELTA) for _ in range(10)]
[move(2, DELTA) for _ in range(10)]
[move(1, DELTA) for _ in range(10)]

IKError: IK failed. Perhaps the distance was between the tip  and target was too large.

# Reach the targets with panda

In [28]:
# We could have made this target in the scene, but lets create one dynamically
target = Shape.create(type=PrimitiveShape.SPHERE,
                      size=[0.05, 0.05, 0.05],
                      color=[1.0, 0.1, 0.1],
                      static=True, respondable=False)

position_min, position_max = [0.6, -0.2, 0.0], [0.8, 0.2, 0.4]

starting_joint_positions = panda.get_joint_positions()

LOOPS = 10
panda.set_joint_positions(starting_joint_positions)
for i in range(LOOPS):

    # Reset the arm at the start of each 'episode'
    #panda.set_joint_positions(starting_joint_positions)

    # Get a random position within a cuboid and set the target position
    pos = list(np.random.uniform(position_min, position_max))
    target.set_position(pos)

    # Get a path to the target (rotate so z points down)
    try:
        path = panda.get_path(
            position=pos, euler=[0, np.radians(180), 0])
    except ConfigurationPathError as e:
        print('Could not find path')
        print("For pos:", pos)
        continue

    # Step the simulation and advance the agent along the path
    done = False
    while not done:
        done = path.step()
        pr.step()
    print('Reached target %d!' % i)

Reached target 0!
Reached target 1!
Reached target 2!
Reached target 3!
Reached target 4!
Reached target 5!
Reached target 6!
Reached target 7!
Reached target 8!
Reached target 9!


# Add object and set the texture

In [29]:
object = Shape.create(type=PrimitiveShape.CUBOID, size=[0.5, 0.5, 0.5], position=[1.0,0.0,0.5], orientation=[0.,0.,0.])
shape1,texture1 = pr.create_texture(filename=TEXTURES+'wood.jpg')
object.set_texture(texture=texture1, mapping_mode=TextureMappingMode.CUBE, repeat_along_u=True, repeat_along_v=True)

for i in range(20):
    pr.step() # simulate behaviour

RuntimeError: The call failed on the V-REP side. Return value: -1

# Quit application

In [ ]:
pr.stop()  # Stop the simulation
pr.shutdown()  # Close the application

In [2]:
import numpy as np
from pyrep import PyRep
from pyrep.objects.shape import Shape
from pyrep.const import PrimitiveShape, TextureMappingMode, JointMode
from pyrep.robots.arms.panda import Panda
from pyrep.robots.end_effectors.panda_gripper import PandaGripper
from pyrep.errors import ConfigurationPathError
import os

In [3]:
SCENES='/'.join(os.getcwd().split('/')[:-1])+str('/include/scenes/')

pr = PyRep()
# Launch the application with a scene file in headless mode
pr.launch(SCENES+'scene_panda_franka_gripper.ttt', headless=False, responsive_ui=True)
pr.start()  # Start the simulation
pr.step()

panda = Panda()  # Get the panda from the scene
gripper = PandaGripper()  # Get the panda gripper from the scene

In [4]:
''' Joint mode enabled '''
panda.get_joint_modes()

[<JointMode.FORCE: 5>,
 <JointMode.FORCE: 5>,
 <JointMode.FORCE: 5>,
 <JointMode.FORCE: 5>,
 <JointMode.FORCE: 5>,
 <JointMode.FORCE: 5>,
 <JointMode.FORCE: 5>]

In [5]:
''' (1) FORCE mode, use: set_joint_target_positions '''
panda.set_joint_target_positions([0.,0.,0.,-np.pi/2,0.,np.pi/3,np.pi])
for i in range(500): pr.step()

panda.set_joint_target_positions([0.,0.,0.,0.,0.,0.,0.])
for i in range(50): pr.step()

panda.set_joint_target_positions([0.,0.,0.,-np.pi/2,0.,np.pi/3,np.pi])
for i in range(50): pr.step()

In [6]:
''' (2) Get is_motor_enabled, motor locked and control loop enabled for every joint '''
[panda.joints[j].is_motor_enabled() for j in range(7)]
[panda.joints[j].is_motor_locked_at_zero_velocity() for j in range(7)]
[panda.joints[j].is_control_loop_enabled() for j in range(7)]

''' Set those properties '''
for i in range(7):
    panda.joints[i].set_motor_enabled(1)
    panda.joints[i].set_motor_locked_at_zero_velocity(0)

''' Actuate single joint '''
panda.joints[0].set_joint_target_position(1.)
for i in range(50): pr.step()    

In [7]:
''' (4) Set maximum forces '''
panda.set_joint_forces([87,	87,	87,	87, 12,	12,	12])
panda.set_joint_target_positions([0.,0.,0.,-np.pi/2,0.,np.pi/3,np.pi])
for i in range(50): pr.step()

''' Limit first joint force to 10Nm '''
panda.set_joint_forces([10,	40,	40,	40, 12,	12,	12])
panda.set_joint_target_positions([1.,0.,0.,-np.pi/2,0.,np.pi/3,np.pi])
for i in range(50): pr.step()

In [8]:
JointMode.__dict__
JointMode

panda.set_joint_mode(JointMode.FORCE)

panda.get_joint_target_positions()
panda.get_joint_target_velocities()

panda.get_joint_velocities()

[1.812744140625,
 0.008654594421386719,
 2.384185791015625e-05,
 0.0002384185791015625,
 0.0005483627319335938,
 0.0027179718017578125,
 9.5367431640625e-05]

In [9]:
''' (3) Get upppper velocity limits [rad/s] '''
panda.get_joint_upper_velocity_limits()

[2.174999952316284,
 2.174999952316284,
 2.174999952316284,
 2.174999952316284,
 2.609999895095825,
 2.609999895095825,
 2.609999895095825]